In [10]:
import os
from dotenv import load_dotenv, find_dotenv
import boto3
import psycopg2
import pandas as pd

#### Chargement des variables d’environnement, configuration AWS et Postgres, et téléchargement du fichier depuis S3

Dans cette section, on charge toutes les variables sensibles (clés AWS, identifiants de base de données) 
depuis le fichier `.env`.  

- Les **identifiants AWS** servent à se connecter à un bucket S3 pour télécharger le fichier final 
  (`hotels_weather_final_ter.csv`).  
- Les **paramètres Postgres** seront utilisés plus tard pour établir une connexion à la base de données.  
- Enfin, on utilise `boto3` pour créer un client S3 et rapatrier le fichier traité depuis le bucket, 
afin de l’exploiter localement.  


In [ ]:
# Load env
env_path = find_dotenv()
load_dotenv(dotenv_path=env_path, override=True)

# control
print(".env chargé depuis :", env_path)
print("USERNAME =", os.getenv("USERNAME"))


.env chargé depuis : c:\Users\Floriane\Desktop\projets_Jedha\bloc1\recap\project_Kayak\.env
USERNAME = flodussart


In [ ]:
# --- AWS credentials (from .env) ---
# Keep these values secret
AWS_ACCESS_KEY_ID = os.getenv('AWS_KEY')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_KEY')

#  Postgres connection params (from .env)
DB_USER = os.getenv('USERNAME')
DB_PASS = os.getenv('PASSWORD')
DB_HOST = os.getenv('HOSTNAME')
DB_PORT = 5432
DB_NAME = os.getenv('DBNAME')

#  S3 settings
S3_BUCKET = 'flodussartprojectkayak'
S3_FILE_KEY = 'projectKayack/src/final_ter.csv'
LOCAL_FILE = 'data/hotels_weather_final_ter.csv'


In [ ]:
# S3 download step
# - Creates a low-level S3 client with explicit credentials and region.
# - Downloads the processed CSV from <bucket>/<key> to LOCAL_FILE on disk.
s3 = boto3.client(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name='eu-north-1'
)
s3.download_file(S3_BUCKET, S3_FILE_KEY, LOCAL_FILE)
print(f"Fichier téléchargé : {LOCAL_FILE}")

Fichier téléchargé : hotels_weather_final_ter.csv


In [14]:
df = pd.read_csv(LOCAL_FILE, encoding='utf-8-sig')
df.head()

,hotel_id,hotel_name,hotel_url,hotel_rating,hotel_description,hotel_latitude,hotel_longitude,city_id,city_name,city_latitude,city_longitude,temp_max,humidity,wind_speed,clouds,pop,good_weather_score
0,1,Aigues Marines,https://www.booking.com/hotel/fr/aigues-marine...,9.5,L’établissement Aigues Marines vous accueille ...,43.559720,4.218698,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
1,2,Appartements 3 étoiles terrasse ou patio intra...,https://www.booking.com/hotel/fr/appartement-3...,9.6,L’établissement Appartements 3 étoiles terrass...,43.565358,4.192750,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
2,3,Artemia Aigues-Mortes - Hotel avec piscine,https://www.booking.com/hotel/fr/le-royal-hote...,9.1,Featuring free WiFi and a seasonal outdoor swi...,43.576396,4.197818,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
3,4,Au Cœur des Remparts,https://www.booking.com/hotel/fr/au-coeur-des-...,9.9,L’hébergement Au Cœur des Remparts se trouve à...,43.565401,4.192973,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714
4,5,Chez Céline et Sébastien,https://www.booking.com/hotel/fr/chez-celine-e...,9.4,L’hébergement Chez Céline et Sébastien se situ...,43.570192,4.195081,1,Aigues-Mortes,43.566152,4.19154,19.544286,53.714286,5.901429,29.571429,1.0,6.610714


#### Chargement des données dans PostgreSQL

Dans cette section, on insère le fichier final (`hotels_weather_final_ter.csv`) dans la base de données PostgreSQL.  

Les étapes sont :  
1. **Connexion à PostgreSQL** avec `psycopg2`, en utilisant les identifiants stockés dans le fichier `.env`.  
2. **Réinitialisation de la table `hotels_weather`** avec `TRUNCATE`, ce qui supprime toutes les anciennes données tout en conservant la structure et les index.  
3. **Chargement en masse des données (COPY)** :  
   - Ouverture du fichier CSV local (encodage `utf-8-sig`).  
   - Suppression de l’en-tête (première ligne).  
   - Utilisation de la commande `COPY FROM STDIN` pour insérer efficacement les données dans la table en respectant l’ordre exact des colonnes.  
4. **Validation et fermeture** : une fois l’import terminé, la transaction est validée (`commit`) et la connexion (curseur et session) est fermée proprement.  

Cette approche optimise l’import de gros volumes de données et garantit que la table `hotels_weather` contient les informations actualisées, prêtes à être exploitées pour des analyses et visualisations.


In [ ]:
# PostgreSQL connection (psycopg2)
# - Uses env vars for host, port, dbname, user, password.
# - Creates a cursor for subsequent SQL commands.
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    dbname=DB_NAME,
    user=DB_USER,
    password=DB_PASS
)
cur = conn.cursor()

In [ ]:
# Reset target table before reloading
# - TRUNCATE removes all rows but keeps the table schema and indexes
cur.execute("TRUNCATE TABLE hotels_weather;")
conn.commit()

In [ ]:
# Bulk load data into PostgreSQL
# - Opens the local CSV (utf-8-sig) and skips the header line.
# - COPY FROM STDIN streams rows efficiently into 'hotels_weather'.
# - Column list in COPY must match the CSV column order exactly.
with open(LOCAL_FILE, 'r', encoding='utf-8-sig') as f:
    next(f)
    cur.copy_expert("""
        COPY hotels_weather (
            hotel_id, hotel_name, hotel_url, hotel_rating, hotel_description,
            hotel_latitude, hotel_longitude, city_id, city_name,
            city_latitude, city_longitude, temp_max, humidity,
            wind_speed, clouds, pop, good_weather_score
        )
        FROM STDIN WITH CSV
    """, f)

In [ ]:
# Finalize the load
# - Commit the transaction and close cursor/connection cleanly.
conn.commit()
cur.close()
conn.close()